## Toxic Comments Classification - Kristie Wirth

In [24]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
import spacy
import string

ImportError: No module named spacy

In [3]:
# Force pandas & numpy to display all data
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
np.set_printoptions(threshold=np.nan)

In [4]:
# Read in data
df = pd.read_csv('../data/train.csv')

In [5]:
# Check the overall shape of the data
df.shape

(159571, 8)

In [6]:
# Looking at data
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [7]:
# Looking at columns & types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 8 columns):
id               159571 non-null object
comment_text     159571 non-null object
toxic            159571 non-null int64
severe_toxic     159571 non-null int64
obscene          159571 non-null int64
threat           159571 non-null int64
insult           159571 non-null int64
identity_hate    159571 non-null int64
dtypes: int64(6), object(2)
memory usage: 9.7+ MB


In [15]:
# Getting ratios for multiple target columns
# Looks like there are strongly imbalanced classes that need to be addressed
df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].apply(pd.Series.value_counts)/df.shape[0]

,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0.904156,0.990004,0.947052,0.997004,0.950636,0.991195
1,0.095844,0.009996,0.052948,0.002996,0.049364,0.008805


In [17]:
# Checking for correlations
# Looks like a lot of comments fall under multiple categories
c = df.corr().abs()
s = c.unstack()
s.sort_values(ascending=False)

identity_hate  identity_hate    1.000000
insult         insult           1.000000
severe_toxic   severe_toxic     1.000000
obscene        obscene          1.000000
threat         threat           1.000000
toxic          toxic            1.000000
insult         obscene          0.741272
obscene        insult           0.741272
toxic          obscene          0.676515
obscene        toxic            0.676515
insult         toxic            0.647518
toxic          insult           0.647518
obscene        severe_toxic     0.403014
severe_toxic   obscene          0.403014
insult         severe_toxic     0.375807
severe_toxic   insult           0.375807
insult         identity_hate    0.337736
identity_hate  insult           0.337736
toxic          severe_toxic     0.308619
severe_toxic   toxic            0.308619
identity_hate  obscene          0.286867
obscene        identity_hate    0.286867
identity_hate  toxic            0.266009
toxic          identity_hate    0.266009
severe_toxic   i

In [19]:
# Looking at a few of the insult labeled comments
df[df['insult']==1].head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
42,001810bf8c45bf5f,You are gay or antisemmitian? \n\nArchangel WH...,1,0,1,0,1,1
43,00190820581d90ce,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",1,0,1,0,1,0
55,0020e7119b96eeeb,Stupid peace of shit stop deleting my stuff as...,1,1,1,0,1,0
56,0020fd96ed3b8c8b,=Tony Sidaway is obviously a fistfuckee. He lo...,1,0,1,0,1,0
